# Calculating the potential revenue of each market

In [31]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [32]:
df = pd.read_csv('../data/cleaned_data/eu_da_prices_clean.csv', index_col=0)
df.date = pd.to_datetime(df.date)
df = df.loc[df.date >= pd.to_datetime('2022-01-01')]
df.head()

,date,hour,datetime,De-Lu,De-Lu Neighbours,Belgium,Denmark 1,Denmark 2,France,Netherlands,Norway 2,Austria,Poland,Sweden 4,Switzerland,Czech,Italy,Slovenia,Hungary,GB
26304,2022-01-01,1,2022-01-01 00:00:00,50.05,78.44,82.02,50.05,46.60,89.06,124.70,132.89,61.84,59.03,46.60,119.97,50.05,170.28,61.84,61.84,81.87
26305,2022-01-01,2,2022-01-01 01:00:00,41.33,69.75,67.07,41.33,41.33,78.48,124.70,129.30,51.44,49.37,41.33,101.54,41.33,155.72,51.44,41.33,64.49
26306,2022-01-01,3,2022-01-01 02:00:00,43.22,68.04,75.11,43.22,42.18,85.16,134.00,132.08,55.91,43.22,42.18,52.13,43.22,147.09,55.91,43.22,24.99
26307,2022-01-01,4,2022-01-01 03:00:00,45.46,51.27,50.91,45.46,44.37,50.00,58.80,111.44,46.94,45.46,44.37,20.78,45.46,91.00,46.94,45.46,7.02
26308,2022-01-01,5,2022-01-01 04:00:00,37.67,42.46,37.67,37.67,37.67,37.67,37.67,112.35,37.67,37.67,37.67,15.66,37.67,104.00,37.67,37.67,2.74


### Single cycle 1 hour

In [77]:
cycle_1_hour_1 = df.drop(['hour', 'datetime'], axis=1).groupby('date').apply(lambda x: x.max() - x.min())
cycle_1_hour_1.head()

,date,De-Lu,De-Lu Neighbours,Belgium,Denmark 1,Denmark 2,France,Netherlands,Norway 2,Austria,Poland,Sweden 4,Switzerland,Czech,Italy,Slovenia,Hungary,GB
date,,,,,,,,,,,,,,,,,,
2022-01-01,0 days,112.30,107.41,112.30,112.30,112.30,112.30,112.30,39.27,112.30,112.30,112.30,133.23,112.30,146.67,112.30,112.30,225.96
2022-01-02,0 days,40.98,56.86,40.53,65.84,65.84,40.53,80.34,31.91,65.84,44.90,65.84,110.41,65.84,145.00,65.84,215.44,143.45
2022-01-03,0 days,149.54,137.56,163.95,139.47,130.33,180.30,215.05,31.49,141.42,100.65,119.75,223.51,141.42,216.73,285.29,173.00,186.21
2022-01-04,0 days,92.44,93.44,91.90,98.53,148.78,85.00,88.90,54.50,90.53,96.03,148.78,115.05,92.44,96.21,90.53,143.64,132.33
2022-01-05,0 days,72.71,113.59,165.17,98.56,98.56,189.62,239.87,44.72,104.18,53.37,98.56,151.03,94.62,189.62,104.18,104.18,116.59


In [81]:
cycle_1_hour_1_rev = cycle_1_hour_1.sum()
cycle_1_hour_1_rev = cycle_1_hour_1_rev.drop('date').astype(float)
cycle_1_hour_1_rev

De-Lu               68252.06
De-Lu Neighbours    54973.29
Belgium             72030.46
Denmark 1           65157.15
Denmark 2           70884.84
France              63843.47
Netherlands         74865.43
Norway 2            38806.75
Austria             59681.02
Poland              52234.22
Sweden 4            73867.40
Switzerland         46144.21
Czech               66943.99
Italy               58954.95
Slovenia            67368.71
Hungary             74368.99
GB                  65669.67
dtype: float64

In [83]:
fig = px.bar(
    cycle_1_hour_1_rev,
    labels=dict(value='€/MW', index=''),
    text_auto='.3s',
    color=cycle_1_hour_1_rev,
    color_continuous_scale='solar'
    )

fig.update_coloraxes(showscale=False)
fig.update_layout(
    title='2022 potential revenue for a single cycle 1 hour battery',
    template='plotly_white',
    xaxis_categoryorder='total descending',
    width=1200,
    height=400
)

fig.write_image('../figures/single-cycle-1-hour-rev.jpeg', scale=5, engine='orca')
fig

### Dual cycle 1 hour

In [95]:
cycle_2_hour_1 = df.copy()
cycle_2_hour_1['am_pm'] = cycle_2_hour_1.hour.apply(lambda x: 'am' if x < 13 else 'pm')
cycle_2_hour_1_am = cycle_2_hour_1.loc[cycle_2_hour_1.am_pm == 'am']
cycle_2_hour_1_pm = cycle_2_hour_1.loc[cycle_2_hour_1.am_pm == 'pm']

In [96]:
cycle_2_hour_1 = cycle_2_hour_1.drop(['hour', 'datetime'], axis=1).groupby(['date', 'am_pm']).apply(lambda x: x.max() - x.min())
cycle_2_hour_1

De-Lu  De-Lu Neighbours  Belgium  Denmark 1  Denmark 2  \
date       am_pm                                                            
2022-01-01 am      46.43             42.53    46.43      46.43      46.43   
           pm      64.81             56.64    64.81      66.86      66.86   
2022-01-02 am      40.53             40.13    40.53      40.53      40.53   
           pm      35.43             46.76    36.57      62.47      62.47   
2022-01-03 am     123.98            114.75   103.16     122.83     113.69   
...                  ...               ...      ...        ...        ...   
2022-12-29 pm      49.42             37.01    51.91      49.42      13.95   
2022-12-30 am      28.51             34.99    29.09      28.51      28.51   
           pm      30.78             40.16    30.81      30.78      30.77   
2022-12-31 am       2.29             12.73     1.54       6.83       4.10   
           pm       3.78             28.42    27.80      63.26      21.81   

                  France  Netherlands  Norway 2  Austria  Poland  Sweden 4  \
date       am_pm                                                             
2022-01-01 am      51.39        96.33     21.45    46.43   46.43     46.43   
           pm      64.81        64.81     33.54    64.81   64.81     66.86   
2022-01-02 am      40.53        40.53     17.88    40.53   40.62     40.53   
           pm      34.92        76.97     31.16    62.47   38.36     62.47   
2022-01-03 am     180.30       130.95     11.88   124.40   99.62    103.11   
...                  ...          ...       ...      ...     ...       ...   
2022-12-29 pm      46.37        85.79     32.87    48.06   17.02     12.23   
2022-12-30 am      38.40        28.65     62.83    54.63   35.58     28.51   
           pm      30.16        30.88    120.55    50.02   54.29     30.77   
2022-12-31 am       1.06         6.82     60.89    10.28   74.39      4.10   
           pm      20.37        93.39     11.41    15.36   74.18     21.81   

                  Switzerland   Czech   Italy  Slovenia  Hungary      GB  
date       am_pm                                                          
2022-01-01 am          104.31   46.43  102.29     46.43    46.43   90.31  
           pm           78.17   64.81  114.66     64.81    64.81  143.62  
2022-01-02 am           72.67   40.53   75.08     40.53    40.53   99.42  
           pm           77.49   62.47  100.21     62.47   212.07  110.07  
2022-01-03 am          214.78  128.46  156.24    232.44   134.01   70.73  
...                       ...     ...     ...       ...      ...     ...  
2022-12-29 pm           37.51   45.18  116.00     70.15    80.04  140.46  
2022-12-30 am           32.40   45.97  106.14     53.39    64.00  140.50  
           pm           31.00   46.49   99.64     72.56   154.47  158.00  
2022-12-31 am           10.00    4.01   78.00      9.29    21.69  149.66  
           pm           15.68   29.80  145.81     46.86   234.51  173.06  

[730 rows x 17 columns]

In [74]:
cycle_2_hour_1_rev = cycle_2_hour_1.sum()
cycle_2_hour_1_rev

De-Lu               104827.20
De-Lu Neighbours     84288.07
Belgium             112553.58
Denmark 1            97049.84
Denmark 2           105492.89
France               93747.97
Netherlands         118202.57
Norway 2             58146.63
Austria              91186.56
Poland               76592.56
Sweden 4            112972.29
Switzerland          69728.75
Czech               104274.78
Italy                86987.76
Slovenia            100646.10
Hungary             111015.74
GB                   96427.40
dtype: float64

In [84]:
fig = px.bar(
    cycle_2_hour_1_rev,
    labels=dict(value='€/MW', index=''),
    text_auto='.3s',
    color=cycle_2_hour_1_rev,
    color_continuous_scale='solar'
    )

fig.update_coloraxes(showscale=False)
fig.update_layout(
    title='2022 potential revenue for a dual cycle 1 hour battery',
    template='plotly_white',
    xaxis_categoryorder='total descending',
    width=1200,
    height=400
)

fig.write_image('../figures/dual-cycle-1-hour-rev.jpeg', scale=5, engine='orca')
fig

### Single cycle 2 hour

(Dis)charging times based on average price profile

In [277]:
charge_3_4 = ['Poland', 'Sweden 4']
charge_4_5 = ['De-Lu', 'De-Lu Neighbours', 'Denmark 2', 'France', 'Norway 2', 'Austria', 'Switzerland', 'Czech', 'Italy', 'Slovenia', 'Hungary', 'GB']
charge_14_15 = ['Belgium', 'Denmark 1', 'Netherlands']

discharge_18_19 = ['GB']
discharge_19_20 = ['Denmark 2', 'Norway 2', 'Sweden 4']
discharge_20_21 = ['De-Lu', 'De-Lu Neighbours', 'Belgium', 'Denmark 1', 'France', 'Netherlands', 'Austria', 'Poland', 'Switzerland', 'Czech', 'Italy', 'Slovenia', 'Hungary']

df['charge_discharge'] = 0

def calculate_single_cycle_2_hour_rev(df):
    charge_3_4_cost = df[charge_3_4].loc[(df.hour == 3) | (df.hour == 4)].sum()
    charge_4_5_cost = df[charge_4_5].loc[(df.hour == 4) | (df.hour == 5)].sum()
    charge_14_15_cost = df[charge_14_15].loc[(df.hour == 14) | (df.hour == 15)].sum()
    discharge_18_19_rev = df[discharge_18_19].loc[(df.hour == 18) | (df.hour == 19)].sum()
    discharge_19_20_rev = df[discharge_19_20].loc[(df.hour == 19) | (df.hour == 20)].sum()
    discharge_20_21_rev = df[discharge_20_21].loc[(df.hour == 20) | (df.hour == 21)].sum()

    charge_cost = pd.concat([charge_3_4_cost, charge_4_5_cost, charge_14_15_cost], axis=0)
    discharge_rev = pd.concat([discharge_18_19_rev, discharge_19_20_rev, discharge_20_21_rev], axis=0)

    total_rev = discharge_rev - charge_cost

    return total_rev

cycle_1_hour_2_rev = calculate_single_cycle_2_hour_rev(df)
cycle_1_hour_2_rev

Austria              85339.86
Belgium              86047.09
Czech                89933.57
De-Lu                83651.41
De-Lu Neighbours     78160.33
Denmark 1            73885.56
Denmark 2            81789.71
France               90023.46
GB                  102684.70
Hungary             109157.07
Italy                85073.47
Netherlands          93562.84
Norway 2             41564.46
Poland               71850.68
Slovenia            100491.01
Sweden 4             98068.84
Switzerland          71004.23
dtype: float64

In [278]:
fig = px.bar(
    cycle_1_hour_2_rev,
    labels=dict(value='€/MW', index=''),
    text_auto='.3s',
    color=cycle_1_hour_2_rev,
    color_continuous_scale='solar'
    )

fig.update_coloraxes(showscale=False)
fig.update_layout(
    title='2022 potential revenue for a single cycle 2 hour battery',
    template='plotly_white',
    xaxis_categoryorder='total descending',
    width=1200,
    height=400
)

fig.write_image('../figures/single-cycle-2-hour-rev.jpeg', scale=5, engine='orca')
fig

### Dual cycle 2 hour

(Dis)charging times based on average price profile

In [312]:
charge_3_4 = ['Poland', 'Sweden 4']
charge_4_5 = ['De-Lu', 'De-Lu Neighbours', 'Denmark 2', 'France', 'Norway 2', 'Austria', 'Switzerland', 'Czech', 'Italy', 'Slovenia', 'Hungary', 'GB', 'Belgium', 'Denmark 1', 'Netherlands']
charge_14_15 = ['Belgium', 'Denmark 1', 'Netherlands', 'De-Lu', 'De-Lu Neighbours', 'Denmark 2', 'Norway 2', 'Austria', 'Poland', 'Sweden 4', 'Switzerland', 'Czech', 'Italy', 'Slovenia', 'Hungary', 'GB']
charge_15_16 = ['France']

discharge_8_9 = ['De-Lu', 'De-Lu Neighbours', 'Belgium', 'Denmark 1', 'Denmark 2', 'Netherlands', 'Austria', 'Poland', 'Sweden 4', 'Switzerland', 'Czech', 'Slovenia', 'Hungary', 'GB']
discharge_9_10 = ['France', 'Norway 2', 'Italy']
discharge_18_19 = ['GB']
discharge_19_20 = ['Denmark 2', 'Norway 2', 'Sweden 4']
discharge_20_21 = ['De-Lu', 'De-Lu Neighbours', 'Belgium', 'Denmark 1', 'France', 'Netherlands', 'Austria', 'Poland', 'Switzerland', 'Czech', 'Italy', 'Slovenia', 'Hungary']

df['charge_discharge'] = 0

def calculate_single_cycle_2_hour_rev(df):
    charge_3_4_cost = df[charge_3_4].loc[(df.hour == 3) | (df.hour == 4)].sum()
    charge_4_5_cost = df[charge_4_5].loc[(df.hour == 4) | (df.hour == 5)].sum()
    charge_14_15_cost = df[charge_14_15].loc[(df.hour == 14) | (df.hour == 15)].sum()
    charge_15_16_cost = df[charge_15_16].loc[(df.hour == 15) | (df.hour == 16)].sum()
    discharge_8_9_rev = df[discharge_8_9].loc[(df.hour == 8) | (df.hour == 9)].sum()
    discharge_9_10_rev = df[discharge_9_10].loc[(df.hour == 9) | (df.hour == 10)].sum()
    discharge_18_19_rev = df[discharge_18_19].loc[(df.hour == 18) | (df.hour == 19)].sum()
    discharge_19_20_rev = df[discharge_19_20].loc[(df.hour == 19) | (df.hour == 20)].sum()
    discharge_20_21_rev = df[discharge_20_21].loc[(df.hour == 20) | (df.hour == 21)].sum()

    charge_cost = pd.concat([charge_3_4_cost, charge_4_5_cost, charge_14_15_cost, charge_15_16_cost], axis=0).reset_index().groupby('index').sum()
    discharge_rev = pd.concat([discharge_8_9_rev, discharge_9_10_rev, discharge_18_19_rev, discharge_19_20_rev, discharge_20_21_rev], axis=0).reset_index().groupby('index').sum()

    total_rev = discharge_rev - charge_cost

    return total_rev

cycle_2_hour_2_rev = calculate_single_cycle_2_hour_rev(df)
cycle_2_hour_2_rev

,0
index,
Austria,120721.43
Belgium,148824.17
Czech,141544.86
De-Lu,147985.57
De-Lu Neighbours,124151.99
Denmark 1,132450.91
Denmark 2,140135.54
France,129589.58
GB,136589.51


In [327]:
fig = px.bar(
    cycle_2_hour_2_rev,
    labels=dict(value='€/MW', index=''),
    text_auto='.3s',
    color=cycle_2_hour_2_rev.values.reshape(-1),
    color_continuous_scale='solar'
    )

fig.update_coloraxes(showscale=False)
fig.update_layout(
    title='2022 potential revenue for a dual cycle 2 hour battery',
    template='plotly_white',
    xaxis_categoryorder='total descending',
    width=1200,
    height=400
)

fig.write_image('../figures/dual-cycle-2-hour-rev.jpeg', scale=5, engine='orca')
fig